In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [ ]:
data = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')

In [ ]:
data.head()

In [ ]:
#Converting into numpy
data = np.array(data)
m,n = data.shape
np.random.shuffle(data)

#To avoid overfitting we split the train set into train and validation so we can adjust our parameters using the validation set
data_dev = data[0:1000].T #Transpose to make each column an example. Earlier it was each row was an example
Y_dev = data_dev[0]
X_dev = data_dev[1:n]
X_dev = X_dev / 255.

data_train = data[1000:m].T
Y_train = data_train[0]
X_train = data_train[1:n]
X_train = X_train / 255.

In [ ]:
Y_train

In [ ]:
X_train[:,0].shape

In [ ]:
#In this code we implement a neural network with 2 layers. Therefore we need to initialise W1,W2,b1,b2 for the 2 layers.
def init_params():
    W1 = np.random.rand(10,784) - 0.5 #random.randn generates random values between [0,1]. We subtract 0.5 to make the range between [-0.5,0.5]
    b1 = np.random.rand(10,1) - 0.5
    W2 = np.random.rand(10,10) - 0.5
    b2 = np.random.rand(10,1) - 0.5
    return W1,W2,b1,b2

In [ ]:
#We add different activation functions to avoid linearity in our model.

def ReLU(z):
    return np.maximum(0,z) #Better way of writing the function. i.e. if x>0;x else if x<0;0. Here the conversion is done elementwise

def deriv_ReLU(z):
    return z>0 #When z>0, slope is 1 otherwise 0. z>0 gives the bool of whether it is True or False which is 0 or 1

def softmax(z):
    A = np.exp(z)/ sum(np.exp(z))
    return A

In [ ]:
#Code for forward propogation
def forward_prop(W1,W2,b1,b2,X):
    z1 = W1.dot(X) + b1
    A1 = ReLU(z1)
    z2 = W2.dot(A1) + b2
    A2 = softmax(z2)
    
    return z1,z2,A1,A2

In [ ]:
#One hot encode the output to give either 0 or 1 for a label
def one_hot(Y):
    one_hot_Y = np.zeros((Y.size,Y.max()+1)) #Create an array of zeros of size of examples * classifications (10 classifications for 10 numbers)
    one_hot_Y[np.arange(Y.size),Y] = 1 #Iterate through each example and if the classification is there, encode as 1
    one_hot_Y = one_hot_Y.T
    return one_hot_Y

In [ ]:
#Code for backward propogation
def back_prop(z1,z2,A1,A2,W2,X,Y):
    m = Y.size
    one_hot_Y = one_hot(Y)
    dz2 = A2 - one_hot_Y
    dw2 = 1/m * dz2.dot(A1.T)
    db2 = 1/m * np.sum(dz2)
    dz1 = W2.T.dot(dz2) * deriv_ReLU(z1)
    dw1 = 1/m * dz1.dot(X.T)
    db1 = 1/m * np.sum(dz1)
    
    return dw1,dw2,db1,db2

In [ ]:
def update_params(W1,W2,b1,b2,dw1,dw2,db1,db2,alpha):
    W1 = W1 - alpha*dw1
    W2 = W2 - alpha*dw2
    b1 = b1 - alpha*db1
    b2 = b2 - alpha*db2
    
    return W1,W2,b1,b2

In [ ]:
def get_predictions(A2):
    return np.argmax(A2,0)

def get_accuracy(predictions,Y):
    print(predictions,Y)
    return np.sum(predictions ==Y)/Y.size

In [ ]:
def gradient_descent(X,Y,iterations,alpha):
    W1,W2,b1,b2 = init_params()
    for i in range(iterations):
        z1,z2,A1,A2 = forward_prop(W1,W2,b1,b2,X)
        dw1,dw2,db1,db2 = back_prop(z1, z2, A1, A2, W2, X, Y)
        W1,W2,b1,b2 = update_params(W1, W2, b1, b2, dw1, dw2, db1, db2, alpha)
        
        if(i%10 == 0):
            print("Iteration: ",i)
            print('Accuracy: ',get_accuracy(get_predictions(A2),Y))
    return W1,W2,b1,b2

In [ ]:
W1,W2,b1,b2 = gradient_descent(X_train,Y_train,500,0.1)

In [ ]:
def make_predictions(W1,W2,b1,b2,X):
    _, _, _, A2 = forward_prop(W1, W2,b1,b2,X)
    predictions = get_predictions(A2)
    return predictions

def test_prediction(index, W1, W2,b1,b2):
    current_image = X_train[:, index, None]
    prediction = make_predictions(W1,W2,b1,b2,X_train[:, index, None])
    label = Y_train[index]
    print("Prediction: ", prediction)
    print("Label: ", label)
    
    current_image = current_image.reshape((28, 28)) * 255
    plt.gray()
    plt.imshow(current_image, interpolation='nearest')
    plt.show()

In [ ]:
test_prediction(16,W1, W2,b1, b2)
test_prediction(59,W1, W2,b1, b2)
test_prediction(92,W1, W2,b1, b2)
test_prediction(106,W1, W2,b1, b2)

In [ ]:
dev_predictions = make_predictions(W1, W2,b1, b2,X_dev)
get_accuracy(dev_predictions, Y_dev)